# Setup

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
import yaml
import os

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

openai = ChatOpenAI(model_name='gpt-4o', temperature=0)

# Sequential Chain

In [2]:
loader = TextLoader('base_conhecimento_britadeira.txt')
documents = loader.load()

historico_conversas = """
Cliente: Minha britadeira não liga.
Chatbot: Você já verificou se a bateria está carregada e conectada corretamente?
"""

pergunta = "Minha britadeira não liga. Eu já verifiquei e a bateria está carregada e conectada corretamente"

In [3]:
inputs = {
    'context' : '\n'.join(doc.page_content for doc in documents),
    'question': pergunta,
    'historico': historico_conversas
}

In [4]:
prompt_base_conhecimento = PromptTemplate(
    input_variables=["context", "question"],
    template="""Use o seguinte contexto para responder à pergunta. 
    Responda apenas com base nas informações fornecidas.
    Não forneceça instruções de procedimento já realizados.
    Não utilize informações externas ao contexto:
    Contexto: {context}
    Pergunta: {question}"""
)
prompt_historico_conversas = PromptTemplate(
    input_variables=["historico", "question"],
    template="""Use o histórico de conversas para responder à pergunta. 
    Responda apenas com base nas informações fornecidas. 
    Não forneceça instruções de procedimento já realizados.
    Não utilize informações externas ao contexto:
    Histórico: {historico}
    Pergunta: {question}"""
)
prompt_final = PromptTemplate(
    input_variables=["resposta_base_conhecimento", "resposta_historico_conversas"],
    template="""Combine as seguintes respostas para gerar uma resposta final,
    mas não forneça instruções de procedimentos já realizados:
    Resposta da base de conhecimento: {resposta_base_conhecimento}
    Resposta do histórico de conversas: {resposta_historico_conversas}"""
)

In [7]:
chain_base_conhecimento = prompt_base_conhecimento | openai
chain_historico_conversas = prompt_historico_conversas | openai
chain_final = prompt_final | openai

In [9]:
resultado_base_conhecimento = chain_base_conhecimento.invoke({'context':inputs['context'], 'question':inputs['question']})
resultado_historico_conversas = chain_historico_conversas.invoke({'historico':inputs['historico'], 'question':inputs['question']})
resultado_final = chain_final.invoke({
    'resposta_base_conhecimento': resultado_base_conhecimento,
    'resposta_historico_conversas': resultado_historico_conversas
})

In [10]:
print(resultado_final.content)

Aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Verifique se o interruptor liga/desliga está na posição correta e se há algum fio solto. Se o problema persistir, contate o suporte técnico pelo 0800 555 5555.
